# Pràctica II - Perceptron amb Scikit-learn

En la sessió anterior hem implementat i entrenat un perceptró des de zero, analitzant pas a pas el seu funcionament intern. Ara, donarem un pas més i treballarem amb la implementació ja disponible a la llibreria scikit-learn, que ens permetrà concentrar-nos en el procés d’aplicació del model a un conjunt de dades real.

Per tal d’il·lustrar-ho, farem servir el problema del Titanic, que ja coneixeu del curs anterior. L’objectiu és predir si un passatger va sobreviure o no al naufragi a partir d’algunes de les seves característiques (edat, sexe, classe del bitllet, etc.).

Tasques a realitzar

1. Carregar i preparar les dades
   - Importar el conjunt de dades del Titanic.
   - Seleccionar les variables més rellevants i realitzar els preprocessaments necessaris (tractament de valors NaNs, codificació de variables categòriques i normalització).
   - Dividir les dades en conjunts d’entrenament i validació.

2. Definir i entrenar el model
   - Utilitzar la classe Perceptron de ``sklearn.linear_model``.
   - Ajustar el model amb les dades d’entrenament.
   - Utilitzar el perceptró implementat la sessió anterior per comparar resultats.

3. Avaluar el rendiment
   - Calcular la *accuracy* sobre les dades de validació.
   - Analitzar la matriu de confusió per entendre millor els encerts i errors del model.
   - Comparar els resultats de ``scikit-learn`` amb la vostra implementació prèvia.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

Podeu trobar les dades del Titanic a [Kaggle](https://www.kaggle.com/competitions/titanic/overview).

In [4]:
df = pd.read_csv('train.csv')

print(df.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


## Neteja de dades

### Codificació de variables categòriques

Farem servir la codificació *one-hot* per a les variables categòriques, ``pandas`` té una funció que ens ho facilita: ``get_dummies()``.

In [5]:
# Codificació one-hot per a les variables categòriques (Sex i Embarked)
df = pd.get_dummies(df, columns=['Sex', 'Embarked'])

### Eliminació de les columnes sense informació

Hi ha algunes columnes que no aporten informació rellevant per a la predicció, com ara el nom del passatger o el número del bitllet. Les hem d'eliminar.

In [6]:
# Eliminacio de columbes innecessaries (Name, Ticket, Cabin, PassengerId)
df = df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])

df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,False,True,False,False,True
1,1,1,38.0,1,0,71.2833,True,False,True,False,False
2,1,3,26.0,0,0,7.9250,True,False,False,False,True
3,1,1,35.0,1,0,53.1000,True,False,False,False,True
4,0,3,35.0,0,0,8.0500,False,True,False,False,True


També fusionarem les columnes *Sex_female* i *Sex_male* amb una sola columna

In [7]:
# Fusionam al columna i cream la columna 'Sex'
df['Sex'] = df['Sex_female']

# Eliminam les columnes innecesaries de sexe
df = df.drop(columns=['Sex_female', 'Sex_male'])

### Normalitzam les dades

Hem de normalitzar les dades per tal que totes les característiques tinguin la mateixa escala. Això és especialment important per a algorismes com el perceptró, que són sensibles a la magnitud de les característiques.

In [8]:
from sklearn.preprocessing import StandardScaler

# Normalitzador de dades
scaler = StandardScaler()

# Normalitzar totes les dades menys la variable objectiu (Survived)
features = [col for col in df.columns if col != 'Survived']

# Normalizam dataframe de totes les variables menys la variables objectiu
df[features] = scaler.fit_transform(df[features])

### Tractament de valors NaN

Els valors NaN els hem de tractar abans d'entrenar el model. Podem optar per eliminar les files amb valors NaN o bé imputar-los amb la mitjana, mediana o moda de la columna. Pandas ens ofereix diverses funcions per fer-ho. Per exemple, podem utilitzar ``fillna()`` per imputar valors, o ``dropna()`` per eliminar files amb valors NaN.

In [9]:
# Miram quines columnes tenen valors nuls
print(df.isnull().sum())

Survived        0
Pclass          0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
Sex             0
dtype: int64


Sabem que *Age* te molts de valors nuls, per tant haurem de rellenar aquests valors nuls amb la mitjana d'edat dels altres valors.

In [10]:
# Eliminam els valors nuls de les columna Age
df['Age'] = df['Age'].fillna(df['Age'].mean())

# Eliminam les files que quedin amb valors nuls
df = df.dropna()

### Divisió del conjunt de dades

Separarem les dades en un conjunt d'entrenament i un de validació. Això ens permetrà avaluar el rendiment del model en dades que no ha vist durant l'entrenament. S'utilitza la funció ``train_test_split()`` de ``sklearn.model_selection``.

In [11]:
X = df.drop(columns=['Survived'])
y = df['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

## Entrenament i avaluació del model

Hem d'entrenar el perceptró amb les dades d'entrenament i després avaluar-lo amb les dades de validació. Hem de fer dos entrenaments, un amb la nostra implementació i un altre amb la de ``scikit-learn``. Per emprar la de ``scikit-learn``, utilitzarem la classe ``Perceptron`` de ``sklearn.linear_model``.

In [12]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, confusion_matrix

# Entrenam el model
model = Perceptron()
model.fit(X_train, y_train)

# Prediccions sobre el conjunt de validació
y_pred = model.predict(X_val)

# Avaluació
accuracy = accuracy_score(y_val, y_pred)
matriu_confusio = confusion_matrix(y_val, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Matriu de confusió:")
print(matriu_confusio)

Accuracy: 0.80
Matriu de confusió:
[[138  19]
 [ 35  76]]
